In [1]:
# check Pillow version number
import PIL
print('Pillow Version:', PIL.__version__)

# load and show an image with Pillow
from PIL import Image

# load and display an image with Matplotlib
import matplotlib.pyplot as plt
from matplotlib import image
from matplotlib import pyplot
import numpy as np
import pandas as pd

import cv2
from skimage import io

import imageio  
import os

# load image and convert to and from NumPy array
from numpy import asarray

Pillow Version: 8.3.1


In [2]:
def find_palette(img):
    
    # Then we can calculate the mean of each chromatic channel following a method analog to the one proposed by @Ruan B.:
    average = img.mean(axis=0).mean(axis=0)
    
    pixels = np.float32(img.reshape(-1, 3))

    n_colors = 5
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
    flags = cv2.KMEANS_RANDOM_CENTERS

    _, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
    _, counts = np.unique(labels, return_counts=True)
    
    dominant = palette[np.argmax(counts)]
    
    avg_patch = np.ones(shape=img.shape, dtype=np.uint8)*np.uint8(average)
    
    indices = np.argsort(counts)[::-1]   
    freqs = np.cumsum(np.hstack([[0], counts[indices]/float(counts.sum())]))
    rows = np.int_(img.shape[0]*freqs)
    
#    dom_patch = np.zeros(shape=img.shape, dtype=np.uint8)
#    for i in range(len(rows) - 1):
#        dom_patch[rows[i]:rows[i + 1], :, :] += np.uint8(palette[indices[i]])
    
    return (dominant, average, palette)

In [ ]:
#path = "../data/train/"
path = "../data/examples/"

mp = np.empty(1, dtype=object)
image_list = []

files = os.listdir(path)

i = 0
for picture in files:
        
    img = io.imread(path+picture)
    #d, a, p = find_palette(image)
    d, a, p = find_palette(img)
    p =  p.reshape(1, 5*3)[0]
    
    image_list.append(picture.split('.')[0])
    r = np.append(p,[a,d])
    if len(mp) == 1:
        mp = r
    else: mp = np.vstack([mp, r])
    #print("Palette: ", mp)
    print(i)
    i +=1

#print(mp)
#print(image_list)

In [ ]:
df = pd.DataFrame()
df["image"] = image_list
i = 1
for e in range(0,mp.shape[1]):
    df["col_"+str(i)] = mp[:, e]
    i +=1
#print(df)
df.to_csv("image_pallette_final1.csv", index=False)